# Library Activity Map

I did this mapping project for the William P. Faust Public Library of Westland, Michigan, in 2021. The library Board of Directors was considering possibilities for expansion or renovation of the library's facilities, and among the options discussed were the opening of a second branch or the purchase of a bookmobile to better serve our patrons who lived farther away from the library. To inform this process, I compiled geographic data from our records and created a map of where the patrons who were actively using the library lived.

## Data Sources

### Patron Activity and Residential Data

Before constructing the map, I pulled the residential addresses for all current library cards in our system. It was determined that simply using all current library card holders would not be demonstrative for this project because, at the beginning of the pandemic, the library had renewed every single library card in its system regardless of how long it had been inactive to allow all patrons to access our online collections or use curbside pickup without having to enter the building to have their card renewed, so this would not provide a useful metric for who was actually using the library.

After examining the data we had available to us, I settled on this operational definition of an active patron as someone who had a library card and had done one of the following within the past year:

* Checked out an item from the library, or
* Accessed an item in one of our digital collections, such as an ebook, audiobook, or streaming video, or
* Used the library's printing system with their library card.

Some important library activities, such as using a public computer or attending an event, could not be measured since the library did not retain identifiable data for these activities.

After consolidating the data from the library's circulation system, digital content services, and print management system, I used a public geocoding API to generate latitude and longitude coordinates for each patron's residence. These steps have been omitted from the report to protect patron privacy, and the csv file of geographic coordinates referenced below will be excluded from the repository.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
# read in latitude and longitude patron_coordinates
patron_coordinates = pd.read_csv('input/patron_coordinates.csv', names=['lat', 'lon'])
# convert to GeoDataFrame
patron_coordinates = gpd.GeoDataFrame(patron_coordinates, geometry=gpd.points_from_xy(patron_coordinates.lon, patron_coordinates.lat))
patron_coordinates.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5787 entries, 0 to 5786
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   lat       5787 non-null   float64 
 1   lon       5787 non-null   float64 
 2   geometry  5787 non-null   geometry
dtypes: float64(2), geometry(1)
memory usage: 135.8 KB


### List of census tracts in city of Westland
compiled from https://www.cityofwestland.com/DocumentCenter/View/1907/Westland-CDBG-Eligible-Census-Tracts-PDF

In [3]:
westland_tracts = [565200, 565301, 565302, 567400, 567300, 567201, 567202, 567100, 567800, 568800, 568900, 565100, 
                   565600, 565700, 568000, 567900, 568200, 568300, 568400, 565900, 565800, 568500, 568700, 567000]

### Michigan Census Block Group Shapefiles
https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html

In [4]:
block_groups = gpd.read_file('input/mich_blockgroups_2020/tl_2020_26_bg.shp')
block_groups.columns = [col.lower().strip() for col in block_groups.columns]
block_groups.head()

,statefp,countyfp,tractce,blkgrpce,geoid,namelsad,mtfcc,funcstat,aland,awater,intptlat,intptlon,geometry
0,26,161,410400,3,261614104003,Block Group 3,G5030,S,1435618,0,+42.2388093,-083.6548830,"POLYGON ((-83.66092 42.24448, -83.66091 42.244..."
1,26,161,400800,2,261614008002,Block Group 2,G5030,S,302334,12839,+42.2837840,-083.7371726,"POLYGON ((-83.74227 42.28764, -83.74194 42.287..."
2,26,161,410100,1,261614101001,Block Group 1,G5030,S,476665,0,+42.2516579,-083.6548132,"POLYGON ((-83.66108 42.25412, -83.65951 42.254..."
3,26,161,405400,1,261614054001,Block Group 1,G5030,S,867179,0,+42.2512633,-083.6962663,"POLYGON ((-83.70046 42.25772, -83.69879 42.257..."
4,26,161,416000,1,261614160001,Block Group 1,G5030,S,1202824,6902,+42.2065707,-083.6737519,"POLYGON ((-83.68482 42.20597, -83.68312 42.206..."


In [5]:
block_groups.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8386 entries, 0 to 8385
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   statefp   8386 non-null   object  
 1   countyfp  8386 non-null   object  
 2   tractce   8386 non-null   object  
 3   blkgrpce  8386 non-null   object  
 4   geoid     8386 non-null   object  
 5   namelsad  8386 non-null   object  
 6   mtfcc     8386 non-null   object  
 7   funcstat  8386 non-null   object  
 8   aland     8386 non-null   int64   
 9   awater    8386 non-null   int64   
 10  intptlat  8386 non-null   object  
 11  intptlon  8386 non-null   object  
 12  geometry  8386 non-null   geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 851.8+ KB


In [6]:
# drop unneeded columns
block_groups.drop(['statefp', 'countyfp', 'namelsad', 'mtfcc', 'funcstat'], axis=1, inplace=True)

In [7]:
# filter to census tracts located in Westland
block_groups.tractce = block_groups.tractce.astype(int)
tract_filter = block_groups.tractce.apply(lambda x: x in westland_tracts)
block_groups = block_groups[tract_filter].copy()
block_groups.shape

(68, 8)

#### Definitions of Remaining Fields

* `tractce`: census tract
* `blkgrpce`: block group
* `geoid`: geographic entity code
* `aland`: land area in meters
* `awater`: water area in meters
* `intptlat, intptlon`: latitude and longitude of internal point
* `geometry`: shape data

### Locations of Nearby Libraries

Collected from Google Maps searches

In [8]:
libraries = pd.read_csv('input/library_locations.csv')
libraries

,name,lat,lon
0,Westland Library,42.32864,-83.40064
1,Wayne Library,42.28132,-83.38334
2,Garden City Library,42.33181,-83.35362
3,Canton Library,42.29704,-83.48795
4,Alfred Noble Library,42.36753,-83.36773
5,Livonia Civic Center,42.39622,-83.36813
6,Plymouth Library,42.37150,-83.46718
7,Inkster Library,42.29204,-83.31342
8,Caroline Kennedy Library,42.33186,-83.27825


### 2020 Census Population Estimates, Wayne County, MI

https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files/2020.html#P1

In [9]:
pop = pd.read_csv('input/pop_estimates_2020/DECENNIALPL2020.P1_data_with_overlays_2021-11-30T125425.csv',
                         header=1, usecols=[0,2], names=['geoid', 'population'])
pop.head()

,geoid,population
0,1500000US261635001001,1331
1,1500000US261635001002,2665
2,1500000US261635002001,857
3,1500000US261635002002,2180
4,1500000US261635003001,1005


In [10]:
# shorten geoid to the last 12 characters, which will match the geoid column in block_groups
pop.geoid = pop.geoid.str[-12:]
pop.head()

,geoid,population
0,261635001001,1331
1,261635001002,2665
2,261635002001,857
3,261635002002,2180
4,261635003001,1005


In [11]:
# add population estimates to block_groups
block_groups = block_groups.merge(pop, 'left', 'geoid')
block_groups.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   tractce     68 non-null     int32   
 1   blkgrpce    68 non-null     object  
 2   geoid       68 non-null     object  
 3   aland       68 non-null     int64   
 4   awater      68 non-null     int64   
 5   intptlat    68 non-null     object  
 6   intptlon    68 non-null     object  
 7   geometry    68 non-null     geometry
 8   population  68 non-null     int64   
dtypes: geometry(1), int32(1), int64(3), object(4)
memory usage: 5.0+ KB


### Population Density

In [12]:
# calculate total area of each block group in miles
block_groups['area_mi'] = (block_groups.aland + block_groups.awater) / 2.59e6

# calculate population density in people per square mile
block_groups['pop_per_mi'] = block_groups.population / block_groups.area_mi
block_groups.head(3)

,tractce,blkgrpce,geoid,aland,awater,intptlat,intptlon,geometry,population,area_mi,pop_per_mi
0,568000,1,261635680001,1290488,0,+42.3206287,-083.3987818,"POLYGON ((-83.40861 42.32402, -83.40823 42.324...",947,0.498258,1900.622090
1,565800,1,261635658001,962337,6819,+42.3066886,-083.3561265,"POLYGON ((-83.36434 42.31040, -83.36051 42.310...",1737,0.374192,4642.008098
2,568400,2,261635684002,1283472,0,+42.3028309,-083.3835427,"POLYGON ((-83.38867 42.30996, -83.38611 42.309...",2529,0.495549,5103.430383


## Data Processing

### Active Patrons

In [13]:
# Calculate patron count per block group
block_groups['active_patrons'] = block_groups.geometry.apply(lambda shape: patron_coordinates.within(shape).sum())
block_groups.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   tractce         68 non-null     int32   
 1   blkgrpce        68 non-null     object  
 2   geoid           68 non-null     object  
 3   aland           68 non-null     int64   
 4   awater          68 non-null     int64   
 5   intptlat        68 non-null     object  
 6   intptlon        68 non-null     object  
 7   geometry        68 non-null     geometry
 8   population      68 non-null     int64   
 9   area_mi         68 non-null     float64 
 10  pop_per_mi      68 non-null     float64 
 11  active_patrons  68 non-null     int64   
dtypes: float64(2), geometry(1), int32(1), int64(4), object(4)
memory usage: 6.6+ KB


In [14]:
block_groups.active_patrons.sum()

5750

In [15]:
patron_coordinates.shape

(5787, 3)

37 sets of coordinates did not fall within the provided block group shapes. This may result from inaccuracies at any point in the process: incorrectly entered addresses in the library system, geocoding errors, slight inaccuracies in the shapefiles, etc. These 37 will be excluded from the dataset going forward.

In [16]:
# calculate percentage of active patrons in population per block group
block_groups['patron_percent'] = block_groups.active_patrons / block_groups.population * 100
block_groups.head()

,tractce,blkgrpce,geoid,aland,awater,intptlat,intptlon,geometry,population,area_mi,pop_per_mi,active_patrons,patron_percent
0,568000,1,261635680001,1290488,0,+42.3206287,-083.3987818,"POLYGON ((-83.40861 42.32402, -83.40823 42.324...",947,0.498258,1900.622090,53,5.596621
1,565800,1,261635658001,962337,6819,+42.3066886,-083.3561265,"POLYGON ((-83.36434 42.31040, -83.36051 42.310...",1737,0.374192,4642.008098,110,6.332758
2,568400,2,261635684002,1283472,0,+42.3028309,-083.3835427,"POLYGON ((-83.38867 42.30996, -83.38611 42.309...",2529,0.495549,5103.430383,163,6.445235
3,568300,3,261635683003,782368,0,+42.2918019,-083.3927784,"POLYGON ((-83.39960 42.29520, -83.39944 42.295...",1573,0.302073,5207.357663,92,5.848697
4,567900,2,261635679002,650428,0,+42.3136469,-083.3839070,"POLYGON ((-83.38891 42.31719, -83.38737 42.317...",1156,0.251131,4603.184365,67,5.795848


## Mapping Setup

I chose [folium](https://python-visualization.github.io/folium/#) as my mapping tool for its interactivity. This would allow the viewer to zoom in and out from overview to street level as needed for discussion and planning.

In [17]:
import folium as fol

In [18]:
# Format shape data for Folium
shapes = gpd.GeoSeries(block_groups.set_index('geoid')['geometry']).to_json()

In [19]:
# assign marker colors to library locations
libraries['color'] = 'blue'
libraries.loc[libraries.name == 'Westland Library', 'color'] = 'green'
# use library name as index
libraries.set_index('name', inplace=True)
libraries.head(3)

,lat,lon,color
name,,,
Westland Library,42.32864,-83.40064,green
Wayne Library,42.28132,-83.38334,blue
Garden City Library,42.33181,-83.35362,blue


In [20]:
def cho_map(col, legend=''):
    '''Generates a choropleth map using the specified column of 
       block_groups (col) as the numerical data, and the provided 
       string (legend) as the label for the legend.'''
    # create base map
    m = fol.Map(location=[42.316805, -83.370413], zoom_start=12)
    # add choropleth map
    fol.Choropleth(geo_data = shapes,
                   name = 'Choropleth',
                   data = block_groups,
                   columns = ['geoid', col],
                   key_on = 'feature.id',
                   fill_color = 'YlGnBu',
                   fill_opacity = 0.65,
                   line_opacity = 0.25,
                   legend_name = legend,
                   smooth_factor = 0
                  ).add_to(m)
    # add library markers
    for i, r in libraries.iterrows():
        fol.Marker([r.lat, r.lon],
             popup=r.name,
             icon=fol.Icon(icon='book', prefix='fa', color=r.color)).add_to(m)
    return m

## Maps
### Population Density

In [21]:
cho_map('pop_per_mi', 'People per Square Mile')

This map shows the population distribution of Westland. The city's irregular shape has always posed a challenge to the library in community engagement. The Westland Library (marked in green) is well-placed to serve the more densely populated areas to the north and east, but there are large portions of population farther to the south and northeast that are relatively far from the library. This also illustrates the importance of our reciprocal partnerships with neighbor libraries, as in many areas the closest public library is not their home library, but a neighboring city's.

### Library Use

In [22]:
cho_map('patron_percent', 'Active Library Users (%)')

This map shows the proportion of active library users for each census block group. We can see that the rate of libray usage tends to be higher closer to the library, and lower on the southern edge and the two panhandles of the city. Comparing this to the population density map, we can see that some of the lower-use areas are also areas of low population, such as the yellow area to the northeast of the library, which covers the Westland Mall and nearby stores. Other areas of low usage, such as the city's southern edge, do contain some denser residential areas. Several of the areas containing both a dense population and high library usage contain apartment complexes, such as the Landings and Fountain Park.

## Conclusion

These maps can help to provide a better understanding of Westland's geography, population distribution, and library usage. My hope is that they will be useful to the board and administration for planning and decisionmaking, and to library staff for planning outreach and community partnerships.

Possibilities for future research include mapping socioeconomic factors such as income, in order to assess the library's success in reaching marginalized populations or identify focus areas for outreach. It could also be useful to examine these maps alongside local bus routes to help identify accessible locations for a potential branch location or bookmobile route.